# Этап 1: Импорт необходимых библиотек и создание SparkSession


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, round as spark_round

# Create a SparkSession
spark = SparkSession.builder.appName("DataAnalysis").getOrCreate()

24/11/14 19:07:16 WARN Utils: Your hostname, nord-laptop resolves to a loopback address: 127.0.1.1; using 192.168.50.104 instead (on interface wlp0s20f3)
24/11/14 19:07:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 19:07:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Этап 2: Определение класса для обработки данных

In [3]:
# Этап 2: Определение класса для обработки данных
class DataProcessor:
    def __init__(self):
        # Инициализация DataFrame'ов
        self.users_df = None
        self.purchases_df = None
        self.products_df = None

    def load_data(self, users_path, purchases_path, products_path):
        # Загрузка CSV-файлов в DataFrame'ы
        self.users_df = spark.read.csv(users_path, header=True, inferSchema=True)
        self.purchases_df = spark.read.csv(purchases_path, header=True, inferSchema=True)
        self.products_df = spark.read.csv(products_path, header=True, inferSchema=True)

    def clean_data(self):
        # Удаление строк с пропущенными значениями
        self.users_df = self.users_df.dropna()
        self.purchases_df = self.purchases_df.dropna()
        self.products_df = self.products_df.dropna()

    def total_purchases_by_category(self):
        # Вычисление общей суммы покупок по категориям продуктов
        joined_df = self.purchases_df.join(self.products_df, 'product_id')
        total_purchases = joined_df.groupBy('category') \
            .agg(spark_sum(col('quantity') * col('price')).alias('total_sales'))
        return total_purchases

    def purchases_by_category_age_group(self):
        # Вычисление суммы покупок по категориям для возрастной группы 18-25
        users_age_group = self.users_df.filter((col('age') >= 18) & (col('age') <= 25))
        purchases_with_users = self.purchases_df.join(users_age_group, 'user_id')
        joined_df = purchases_with_users.join(self.products_df, 'product_id')
        total_purchases = joined_df.groupBy('category') \
            .agg(spark_sum(col('quantity') * col('price')).alias('total_sales'))
        return total_purchases

    def percentage_purchases_by_category_age_group(self):
        # Вычисление процента покупок по категориям для возрастной группы 18-25
        total_purchases_df = self.purchases_by_category_age_group()
        total_sales = total_purchases_df.agg(spark_sum('total_sales').alias('total_sales')).collect()[0]['total_sales']
        percentage_df = total_purchases_df.withColumn('percentage', spark_round((col('total_sales') / total_sales) * 100, 2))
        return percentage_df

    def top_three_categories(self):
        # Выбор 3 категорий с наибольшим процентом расходов среди потребителей в возрасте от 18 до 25 лет
        percentage_df = self.percentage_purchases_by_category_age_group()
        top_three = percentage_df.orderBy(col('percentage').desc()).limit(3)
        return top_three

# Этап 3: Создание экземпляра класса и загрузка данных

In [4]:
processor = DataProcessor()

users_csv_path = 'db/users.csv'
purchases_csv_path = 'db/purchases.csv'
products_csv_path = 'db/products.csv'

# Load data
processor.load_data(users_csv_path, purchases_csv_path, products_csv_path)

# Этап 4: Очистка данных

In [5]:
processor.clean_data()

# Этап 5: Вычисление общей суммы покупок по категориям продуктов

In [6]:
total_purchases = processor.total_purchases_by_category()
total_purchases.show()

+-----------+------------------+
|   category|       total_sales|
+-----------+------------------+
|       Home|1523.4999999999998|
|     Sports|1802.4999999999998|
|Electronics|1174.7999999999997|
|   Clothing|             790.3|
|     Beauty| 459.8999999999999|
+-----------+------------------+



# Этап 6: Вычисление суммы покупок по категориям для возрастной группы 18-25

In [7]:
age_group_purchases = processor.purchases_by_category_age_group()
age_group_purchases.show()

+-----------+------------------+
|   category|       total_sales|
+-----------+------------------+
|       Home|             361.1|
|     Sports|310.49999999999994|
|Electronics|             249.6|
|   Clothing|             245.0|
|     Beauty|41.400000000000006|
+-----------+------------------+



# Этап 7: Вычисление процента покупок по категориям для возрастной группы 18-25

In [8]:
percentage_purchases = processor.percentage_purchases_by_category_age_group()
percentage_purchases.show()

+-----------+------------------+----------+
|   category|       total_sales|percentage|
+-----------+------------------+----------+
|       Home|             361.1|      29.9|
|     Sports|310.49999999999994|     25.71|
|Electronics|             249.6|     20.67|
|   Clothing|             245.0|     20.29|
|     Beauty|41.400000000000006|      3.43|
+-----------+------------------+----------+



# Этап 8: Выбор 3 категорий с наибольшим процентом расходов среди потребителей в возрасте от 18 до 25 лет

In [9]:
top_three_categories = processor.top_three_categories()
top_three_categories.show()

+-----------+------------------+----------+
|   category|       total_sales|percentage|
+-----------+------------------+----------+
|       Home|             361.1|      29.9|
|     Sports|310.49999999999994|     25.71|
|Electronics|             249.6|     20.67|
+-----------+------------------+----------+



In [10]:
# Close the SparkSession
spark.stop()